# Agent Group Chat

## Overview 

Two Agents will be defined: 

- the "Retrieve Winner Agent" and 
- the "Retrieve Score Agent" 

Both Agents are tasked with determining *"Who won the Super Sports Championship 2025"*, a fictional sport event with no available internet data. 
This setup highlights how private data can be leveraged to empower the agents. Consequently, both agents use LLM function calling to identify and execute local functions to retrieve the necessary information to answer the task.

![Overview](../media/img/GroupChat_Overview.png)

As the name indicates, the "Retrieve Winner Agent" will retrieve the winner of the sport event, while the "Retrieve Score Agent" will retrieve the final score.


## Environment

An Azure OpenAI GPT-4 instance will be used by Semantic Kernel and it's current Agent framework to identify which local functions need to be executed. The necessary connection information is imported from the ***config.env*** file in the [../config/ folder](../config). You can use the provided [Azure CLI Powershell script](../setup/setup.azcli) to create the necessary instance.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.38.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");

Console.WriteLine($"Configuration loaded...");

## Plugin Definition

Definition of two mock functions which can be identified by the LLM function call communication used and executed by both Agents. Both functions are provided as class functions to simplify the [Semantic Kernel Plug In](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/?pivots=programming-language-csharp) creation. 

In [2]:
public class RetrieveWinner
{
    [KernelFunction("get_sport_event_winner")]
    //Description is just necessary if function name is not self-explanatory
    [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the winner of the sport event.
        return "Munich Ice Tiger Flying Dolphins";
    }
}

public class RetrieveScore
{
    [KernelFunction("get_sport_event_score")]
    //Description is just necessary if function name is cryptic and not self-explanatory
    [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the result of the sport event.
        return "24:1";
    }
}

Console.WriteLine($"Plugins defined...")

## Agent Definition

The two `ChatCompletionAgent()` instances are created using the [Semantic Kernel - Agents Framework](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/?pivots=programming-language-csharp). 

The `agentInstruction` is simple:
- RetrieveWinner Agent: You can retrieve the winner of a sport event
- RetrieveScore Agent:  You can retrieve the final score of a sport event


In [3]:
#pragma warning disable SKEXP0110, SKEXP0001

//Define RetrieveWinner Agent
IKernelBuilder retrieveWinnerKernelBuilder = Kernel.CreateBuilder();
retrieveWinnerKernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: apiKey, 
    endpoint: endpoint, 
    deploymentName: chatCompletionDeployment
);
retrieveWinnerKernelBuilder.Plugins.AddFromType<RetrieveWinner>("RetrieveWinner");
Kernel retrieveWinnerKernel = retrieveWinnerKernelBuilder.Build();

string agentName = "RetrieveWinner";
string agentInstruction = "You can retrieve the winner of a sport event";

ChatCompletionAgent retrieveWinnerAgent = new ChatCompletionAgent()
{
    Name = agentName,
    Instructions = agentInstruction,
    Kernel = retrieveWinnerKernel,
    Arguments = new KernelArguments(
        new OpenAIPromptExecutionSettings() { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        }
    )
};

//Define retrieveScore Agent
IKernelBuilder retrieveScoreKernelBuilder = Kernel.CreateBuilder();
retrieveScoreKernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: apiKey, 
    endpoint: endpoint, 
    deploymentName: chatCompletionDeployment
);
retrieveScoreKernelBuilder.Plugins.AddFromType<RetrieveScore>("RetrieveScore");
Kernel retrieveScoreKernel = retrieveScoreKernelBuilder.Build();

agentName = "RetrieveScore";
agentInstruction = "You can retrieve the final score of a sport event";

ChatCompletionAgent retrieveScoreAgent = new ChatCompletionAgent()
{
    Name = agentName,
    Instructions = agentInstruction,
    Kernel = retrieveScoreKernel,
    Arguments = new KernelArguments(
        new OpenAIPromptExecutionSettings() { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        }
    )
};

Console.WriteLine($"retrieveWinner Agent created...");
Console.WriteLine($"retrieveScore Agent created...");

## Create Group Chat

A Group chat is created and executed. The chat will be terminated after:

- a custom terminate function `KernelFunction terminateFunction` detects that within the agent communication a sport event and a score is detected. A call to the LLM will be used to execute the function or
- after max 10 agent chat iterations 

In [4]:
#pragma warning disable SKEXP0110, SKEXP0001

KernelFunction terminationFunction = AgentGroupChat.CreatePromptFunctionForStrategy(
    $$$""" 
        Determine if there's a sport event and a score.
        Be precise. There needs to be a sport event and a score. 
        If there's no sport event or no score or both, respond with a single word: no
        If so, respond with a single word: yes

        History:
        {{$history}}
    """,
    safeParameterNames: "history"
);
KernelFunctionTerminationStrategy kernelFunctionTerminationStrategy = 
    new KernelFunctionTerminationStrategy(terminationFunction, retrieveScoreKernel){
        ResultParser = (result) => result.GetValue<string>()?.Contains("yes", StringComparison.OrdinalIgnoreCase) ?? false,
        HistoryVariableName = "history",
        MaximumIterations = 10

    };

AgentGroupChat agentGroupChat = new AgentGroupChat(retrieveWinnerAgent, retrieveScoreAgent);
agentGroupChat.ExecutionSettings = new AgentGroupChatSettings(){
    TerminationStrategy = kernelFunctionTerminationStrategy 
};

Console.WriteLine($"AgentGroupChat with custom termination criteria created...");

## Execute Group Chat

The Agent Group Chat is initiated and the agents start executing.

In [5]:
#pragma warning disable SKEXP0110

string prompt = "Who won the Super Sports Championship 2025. Return name of the winning team and the score.";

agentGroupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, prompt));

string response = "";
await foreach (ChatMessageContent chatMessageContent in agentGroupChat.InvokeAsync())
{
    response = String.Concat(response, chatMessageContent.Content, "\n");
    Console.WriteLine(chatMessageContent.Content);
}

### Agent results

The `retrieveWinner` agent has provided the information (by calling the provided plug-in) that the **Munich Ice Tiger Flying Dolphins** (a fictional team) has won. 

The information that the final score is **24:1** is provided by the `retrieveScore` agent as his contribution to solving the task.